In [2]:
suppressMessages({
    library(extRemes)
})

Options to include:
  - shift & fixed dispersion
  - normal & GEV
  - logged or negative values
  - single covariate

Methods needed:
  - [x] model fitting
  - [x] parameters of nonstationary model
  - [ ] map to u
  - [ ] return periods
  - [ ] return times
  - [ ] return level plot
  - [ ] GMST trend plot
  - [ ] time trend plot
  - [ ] obs results for spreadsheet
  - [ ] model results for spreadsheet

---
## **Fitting methods**

### **Nonstationary log-likelihood**

In [105]:
# nonstationary log-likelihood function
ns_loglik <- function(pars, cov1, x, dist, fittype) {

    # compute nonstationary location & scale
    if(fittype == "fixeddisp") {
        const = exp((pars["alpha"] * cov1) / pars["mu0"])
        loc = pars["mu0"] * const
        scale = pars["sigma0"] * const
    } else if(fittype == "shift") {
        loc = pars["mu0"] + pars["alpha"] * cov1
        scale = pars["sigma0"]
    } else {
        print(paste(fittype, "not implemented"))
        return()
    }
    
    # return negative log-likelihood
    if(dist == "norm") {
        return(-sum(dnorm(x, mean = loc, sd = scale, log = T)))
    } else if(dist == "gev") {
        shape = pars["shape"]
        return(-sum(devd(x, loc = loc, scale = scale, shape = shape, log = T)))
    } else {
        print(paste(dist, "not implemented"))
        return()
    }
}

In [73]:
# load model data for testing
log10_pr <- merge(read.csv("data/test_gmst.dat", comment.char = "#", sep = " ", col.names = c("year", "gmst")),
            read.csv("data/test_precip.dat", comment.char = "#", sep = " ", col.names = c("year", "pr")))
log10_pr$log10_pr <- log10(log10_pr$pr)

tmax <- merge(read.csv("data/test_gmst.dat", comment.char = "#", sep = " ", col.names = c("year", "gmst")),
            read.csv("data/test_tasmax.dat", comment.char = "#", sep = " ", col.names = c("year", "tmax")))

In [93]:
test_nsll <- function(varnm, dist, fittype, dp = 3) {
    # quick testing function
    
    df <- get(varnm)
    init <- c("mu0" = mean(df[,varnm]), "sigma0" = sd(df[,varnm]), "alpha" = 0)
    if(dist == "gev") init <- c(init, "shape" = 0)
    
    fitted <- round(optim(par = init, ns_loglik, cov1 = df$gmst, x = df[,varnm], dist = dist, fittype = fittype)$par, dp)
    if(dist == "norm") fitted <- c(fitted, "shape" = NA)[c("mu0", "sigma0", "shape", "alpha")]
    
    c("variable" = varnm, "dist" = dist, "fittype" = fittype, fitted)
}

In [94]:
rbind(test_nsll("tmax", "gev", "shift"),
      test_nsll("tmax", "gev", "fixeddisp"),
      test_nsll("log10_pr", "norm", "shift"),
      test_nsll("log10_pr", "norm", "fixeddisp"))

variable,dist,fittype,mu0,sigma0,alpha,shape
tmax,gev,shift,32.059,0.962,3.185,-0.095
tmax,gev,fixeddisp,32.11,0.929,3.017,-0.118
log10_pr,norm,shift,2.145,0.176,NA,0.127
log10_pr,norm,fixeddisp,2.145,0.171,NA,0.126


In [88]:
# results from climate explorer - not identical, but very similar
read.csv("cx-results.txt", sep = " ")

variable,dist,fittype,mu0,sigma0,shape,alpha
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
tmax,gev,shift,32.119,0.962,-0.108,3.100
tmax,gev,fixeddisp,32.135,0.914,-0.089,3.102
log10_pr,norm,shift,2.135,0.175,NA,0.141
log10_pr,norm,fixeddisp,2.134,0.169,0.146,NA


### **Wrapper method to fit distribution**

In [177]:
# wrapper to fit nonstationary fixed-dispersion model

fit_ns <- function(dist, type = "fixeddisp", data, varnm, covnm_1, lower = F, mintemps = F, event_index = NA, ...) {
    
    # currently only works for distributions fully specified by mean & sd: only tested for normal, lognormal
    if(! dist %in% c("norm", "gev")) {
        print("Not yet implemented: use norm or gev")
        return()
    }
    
    # if looking at minimum temperatures (or minima of negative values generally), need to flip data for stable model fitting
    x <- data[,varnm]; if(mintemps) x <- -x
    cov1 <- data[,covnm_1]
    
    # fit model with appropriate number of parameters, pad if necessary
    init <- c("mu0" = mean(x), "sigma0" = sd(x), "alpha" = 0)
    if(dist == "gev") init <- c(init, "shape" = 0)
    fitted <- suppressWarnings(optim(par = init, ns_loglik, cov1 = cov1, x = x, dist = dist, fittype = type))
    
    # if looking at minimum temperatures (or minima of negative values generally), so trend & location parameters have been flipped. This may cause some confusion so may have to modify later!
    if(mintemps) {
        fitted[["NOTE"]] <- "NB: model parameters are estimated for negative values"
        fitted$par["mu0"] <- -fitted$par["mu0"]
        fitted$par["alpha"] <- -fitted$par["alpha"]
        x <- -x
    }
            
    # attach assorted useful information
    fitted[["dist"]] <- dist
    fitted[["type"]] <- type
    fitted[["varnm"]] <- varnm
    fitted[["covnm_1"]] <- covnm_1
    fitted[["x"]] <- x
    fitted[["cov1"]] <- data[,covnm_1]
    
    fitted[["lower"]] <- lower               # saves having to specify every time later on
    fitted[["mintemps"]] <- mintemps         # look at maxima of 0-temps, rather than minima of observed temps
    
    if(is.na(event_index)) { event_index <- length(x) } # assume that year of interest is most recent, unless told otherwise (used in later plotting functions)
    fitted[["ev_idx"]] <- event_index

    return(fitted)
}

In [178]:
# load model data for testing
pr <- merge(read.csv("data/test_gmst.dat", comment.char = "#", sep = " ", col.names = c("year", "gmst")),
            read.csv("data/test_precip.dat", comment.char = "#", sep = " ", col.names = c("year", "pr")))
pr$log10_pr <- log10(pr$pr)

tmax <- merge(read.csv("data/test_gmst.dat", comment.char = "#", sep = " ", col.names = c("year", "gmst")),
            read.csv("data/test_tasmax.dat", comment.char = "#", sep = " ", col.names = c("year", "tmax")))

tmin <- merge(read.csv("data/test_gmst.dat", comment.char = "#", sep = " ", col.names = c("year", "gmst")),
              read.csv("data/test_tasmin.dat", comment.char = "#", sep = " ", col.names = c("year", "tmin")))

pr_shift <- fit_ns("norm", "shift", pr, "log10_pr", "gmst", lower = T)
pr_fd <- fit_ns("norm", "fixeddisp", pr, "log10_pr", "gmst", lower = T)

tmax_shift <- fit_ns("gev", "shift", tmax, "tmax", "gmst")
tmax_fd <- fit_ns("gev", "fixeddisp", tmax, "tmax", "gmst")

tmin_shift <- fit_ns("gev", "shift", tmin, "tmin", "gmst", lower = T, mintemps = T)

### **Get nonstationary parameters**

In [179]:
ns_pars <- function(mdl, fixed_cov = NA) {
    
    # if no covariate value given, evaluate at all covariate values
    if(is.na(fixed_cov)) fixed_cov <- mdl$cov1
    
    # calculate the nonstationary parameter values
    pars <- mdl$par
    if(mdl$type == "fixeddisp") {
        
        ns_const = exp((pars["alpha"] * fixed_cov) / pars["mu0"])
        loc = pars["mu0"] * ns_const
        scale = pars["sigma0"] * ns_const
        
    } else if(mdl$type == "shift") {
        loc = pars["mu0"] + pars["alpha"] * fixed_cov
        scale = pars["sigma0"]
        
    } else {
        print(paste(mdl$type,"not implemented"))
        return()
    }
        
    # return the list of named parameters: location, scale, shape (if applicable)
    if("shape" %in% names(pars)) {
        return(lapply(list("loc" = loc, "scale" = scale, "shape" = pars["shape"]), unname))
    } else {
        return(lapply(list("loc" = loc, "scale" = scale), unname))
    }
    
}

In [180]:
ns_pars(tmin_shift, tmin$gmst[tmin$year == 2022])

$loc
[1] 1.99665

$scale
[1] 1.491419

$shape
[1] -0.1208743

In [176]:
tmin$gmst[tmin$year == 2022]

[1] 0.9165972